In [25]:
# scraping libraries to read html code
import requests
from bs4 import BeautifulSoup
import json

In [26]:
# pandas library to create dataframe
import pandas as pd
import numpy as np

In [95]:
# Direct to initial results
url = 'https://www.yelp.com/search?find_desc='
keyword = input('What do you want to eat ? ')
address = input('Where are you ? ')

url = 'https://www.yelp.com/search?find_desc='+ keyword + '&find_loc=' + address 
url

What do you want to eat ? burger
Where are you ? boston


'https://www.yelp.com/search?find_desc=burger&find_loc=boston'

In [96]:
# Checking if the url is "valid", the status code should be 200
response = requests.get(url)
response.status_code

200

In [110]:
# extract the html code from the web page = https://www.yelp.com/search?find_desc=Restaurants&find_loc=New+York,+NY&start=30
results_page = BeautifulSoup(response.content,'lxml')

In [111]:
# creating a list of businesses with the html code of the business as elements of the list
businesses = results_page.find_all('a',class_ = "biz-name js-analytics-click")

In [112]:
# selecting the first element (a particular restaurant in our case) of the list businesses and looking in the
#ignore the first element of the businesses list, it is an ad by yelp, regular list start from the second (index = 1)
# html code for its name
business = businesses[1]
name = business.find('span').get_text()

# getting the link to the yelp page this particular restaurant
link = 'https://www.yelp.com' + business.get('href')

In [113]:
print(name,link)

Boston Burger Company - Boston https://www.yelp.com/biz/boston-burger-company-boston-boston-2?osq=burger


In [114]:
# extracting the html code of the web page associated with the first business of our list businesses
response_business = requests.get(link)
results_business_page = BeautifulSoup(response_business.content,'lxml')

In [21]:
# looking in the html page of the restaurant for latitude and longitude coordinates of the restaurant
# the json object json_map contains these coordinates
json_map = results_business_page.find('div', class_ = "lightbox-map hidden").get('data-map-state')

# we can convert this json object to a python dictionnary using the following method
dict_map = json.loads(json_map)

# the relevant keys in this dict are "markers", "location" and "latitude" and "longitude"
latitude = dict_map["markers"][1]["location"]["latitude"]
longitude = dict_map["markers"][1]["location"]["longitude"]

In [78]:
json_map

'{"serviceAreas": [], "moMapPossible": true, "scrollwheelZoom": false, "zoomControlPosition": "top_right", "minZoomlevel": null, "isFullBleed": false, "maxZoomlevel": null, "zoom": 15, "library": "google", "fitToGeobox": false, "hoods": [], "adPinColor": null, "markers": [{"location": null, "key": "directions_marker", "icon": {"name": "directions", "anchorOffset": [12, 32], "activeOrigin": [0, 0], "scaledSize": [24, 32], "regularUri": "https://s3-media4.fl.yelpcdn.com/assets/srv0/yelp_maps/79f63ebc20db/assets/img/directions@2x.png", "size": [24, 32], "activeUri": "https://s3-media4.fl.yelpcdn.com/assets/srv0/yelp_maps/7249ab345ac8/assets/img/directions_highlighted@2x.png", "regularOrigin": [0, 0]}}, {"resourceType": "business", "url": "/biz/ramen-hood-new-york", "resourceId": "oy_FqW6XO8S06pg6nHbhog", "shouldOpenInNewTab": false, "location": {"latitude": 40.7223112701284, "longitude": -73.9973774072879}, "key": "starred_business", "hovercardId": "xJJUNXIR1JECvREs7j-Mzg", "icon": {"name

In [79]:
print(latitude,longitude)

40.7223112701284 -73.9973774072879


In [80]:
# The category can be found in the 'a' tag under "span", class_ = 'category-str-list' tag, 
category= results_business_page.find("span", class_ = 'category-str-list').find('a').get_text()

In [81]:
category

'Ramen'

In [115]:
# the price range ('$','$$','$$$', or '$$$$') can be found in the "span", class_ = 'business-attribute price-range' tag
# the price range is not always available, that is why we need to catch errors 
# if necessary (hence the try: except) structure

try:
    price_range = results_business_page.find("span", class_ = 'business-attribute price-range').get_text()
except:
    price_range = None

In [116]:
print(price_range)

$$


In [117]:
# the ratings associated to a particular restaurant are in the "div", itemprop = 'aggregateRating' tag
# the rating value (between 0 and 5) can be accessed through ratings (= results_business_page.find("div", 
# itemprop = 'aggregateRating') at the tag "meta" and the attribute 'content'
# the number of reviews is available through ratings at the 'span' tag

ratings = results_business_page.find("div", itemprop = 'aggregateRating')
rating_value = ratings.find("meta").get('content')
review_count = ratings.find("span").get_text()

In [118]:
print(rating_value,review_count)

4.0 801


In [119]:
# the phone number can be accessed at the tag 'span',itemprop = "telephone"
# phone numbers are not always available so we need to catch errors using the try:... except: structure

try:
    phone_number = results_business_page.find('span',itemprop = "telephone").get_text().strip()
except:
    phone_number = None   

In [120]:
print(phone_number)

(857) 233-4560


In [66]:
# We want to know if the restaurant is currently open: this information is 
# given at the 'span', class_ = "nowrap extra open" tag. This information is not always 
# available that is why we use the try: ... except:... structure

try:
    open_ = results_business_page.find('span', class_ = "nowrap extra open").get_text()
except:
    open_ = None

In [67]:
print(open_)

Open now


In [121]:
try:
    addr = results_business_page.find('strong', class_ = "street-address").get_text().strip()
except:
    addr = None
addr

'1100 Boylston StBoston, MA 02215'

In [122]:
try:
    web = results_business_page.find('span', class_ = "biz-website js-biz-website js-add-url-tagging").find('a').get_text()
except:
    web = None
web

'bostonburgerco.com'

In [44]:
#want to put all infomations we want to get from one business into one funtion
#input: specific restaurant url link
#output: all details we tested above
def get_info(link):
    response_business = requests.get(link)
    results_business_page = BeautifulSoup(response_business.content,'lxml')
    
    try:
        addr = results_business_page.find('strong', class_ = "street-address").get_text().strip()
    except:
        addr = None

    try:
        json_map = results_business_page.find('div', class_ = "lightbox-map hidden").get('data-map-state')
        dict_map = json.loads(json_map)
        lat = dict_map["markers"][1]["location"]["latitude"]
        lon = dict_map["markers"][1]["location"]["longitude"]
    except: 
        json_map = None 

    try: 
        category = results_business_page.find("span", class_ = 'category-str-list').find('a').get_text()
    except:
        category = None
        
    try:
        price_range = results_business_page.find("span", class_ = 'business-attribute price-range').get_text()
    except:
        price_range = None
        
    try: 
        rating_value = results_business_page.find("div", {'itemprop' : "aggregateRating"}).find("meta").get('content')
    except: 
        rating_value = None
          
    try:
        phone_number = results_business_page.find('span',itemprop = "telephone").get_text().strip()
    except:
        phone_number = None
        
    try:
        open_ = results_business_page.find('span', class_ = "nowrap extra open").get_text()
    except:
        open_ = 'closed now'
        
    try:
        web = results_business_page.find('span', class_ = "biz-website js-biz-website js-add-url-tagging").find('a').get_text()
    except:
        web = None
    
    try:
        moreinfo = results_business_page.find_all('dl')
        delivery = "No"
        for i in moreinfo[1:]:
            try:
                attr = i.find('dt',{'class':"attribute-key"}).get_text().strip()
                yon = i.find('dd').get_text().strip()
                if attr == 'Delivery':
                    delivery = yon
                    break
            except:
                delivery = delivery

    except:
        delivery = None
        
    return(addr,category,price_range,rating_value,phone_number,open_,web,delivery,lat,lon)
    

In [94]:
#Get a list of url links of all businesses (or maybe the first 30)
#so that we could compile a list of details on, for example, the first 30 search-result restaurants
link_list = []

for i in range(1,len(businesses)):
    business = businesses[i]
    link = 'https://www.yelp.com' + business.get('href')
    link_list.append(link)

In [96]:
#Create a list of restaurant names
name_list=[]
for i in range(1,len(businesses)):
    business = businesses[i]
    name = business.find('span').get_text()
    name_list.append(name)

In [98]:
#Create a list of all restaurants' information
info_list=[]

for i in range(len(link_list)):
    info_list.append(get_info(link_list[i]))

In [73]:
def get_businesses():
    keyword = input('What do you want to eat ? ')
    address = input('Where are you ? ')
    limit = input('How many results do you want? ')
    filters = input('Do you want to sort by price or by rating? ')
    
    #first page - figure out numbers of results per page
    url = 'https://www.yelp.com/search?find_desc='+ keyword + '&find_loc=' + address 
    response = requests.get(url)
    try:
        results_page = BeautifulSoup(response.content,'lxml')
    except:
        print("we didn't get back result page from yelp correctly")
    noperpage = results_page.find('span',{'class':'pagination-results-window'}).get_text().strip()[10:12]
    noperpage = int(noperpage)
    businesses = results_page.find_all('a',{'data-analytics-label':"biz-name"})[1:]
    
    #start loop from the second page to get enough number of restaurants
    loop_range = (int(limit)-1)//noperpage
    for pagination in range(1, loop_range+1):
        url = 'https://www.yelp.com/search?find_desc='+ keyword + '&find_loc=' + address + '&start=' + str(pagination*noperpage)
        response = requests.get(url)
        try:
            results_page = BeautifulSoup(response.content,'lxml')
        except:
            print("we didn't get back result page from yelp correctly")
        
        businesses.extend(results_page.find_all('a',{'data-analytics-label':"biz-name"})[1:])
    businesses = businesses[:int(limit)]
    
    #generate details by get_info and format in dataframe
    business_list = []
    for business in businesses:
        name = business.find('span').get_text()
        link = 'https://www.yelp.com' + business.get('href')
        addr,category,price_range,rating_value,phone_number,open_,web,delivery,lat,lon= get_info(link)
        business_list.append([name,addr,open_,price_range,rating_value,category,phone_number,web,delivery,lat,lon])
    df = pd.DataFrame(business_list[:int(limit)+1], columns = ['name','address','open or closed','price range','rating value','category','phone','website','delivery service','lat','lon'])
    
    #sortby...
    if filters == 'rates':
        df = df.sort_values(by='rating value',ascending=False)
        
    elif filters == 'price':
        df['length'] = df['price range'].apply(lambda x: len(x))
        df = df.sort_values(by='length',ascending=False)
        df = df.drop('length', axis=1)
    
    return(df)

In [74]:
res = get_businesses()

What do you want to eat ? noodles
Where are you ? flushing
How many results do you want? 12
Do you want to sort by price or by rating? rating


In [75]:
res

,name,address,open or closed,price range,rating value,category,phone,website,delivery service,lat,lon
0,Bó Noodle House,"3907 Prince St 1HQueens, NY 11354",closed now,$$,3.5,Chinese,(718) 683-5919,None,No,40.759389,-73.831890
1,Shi Miao Dao Yunan Rice Noodle,"135-25 40th RdFlushing, NY 11354",Open now,$$,4.5,Noodles,(917) 563-5975,None,No,40.758837,-73.830799
2,Dunhuang Lanzhou Beef Noodle,"136-93 37th AveFlushing, NY 11354",Open now,$,4.0,Chinese,(718) 359-2188,None,No,40.762444,-73.827791
3,Beef Stew Chop-Shop,"37-11 Main StBasement Food CourtQueens, NY 11354",closed now,$,4.0,Noodles,,None,No,40.761370,-73.830373
4,Lamb Noodle Soup,"4128 Main StGolden MallFlushing, NY 11355",Open now,$,4.5,Chinese,(516) 234-3100,None,No,40.757168,-73.829317
5,Kun Sohn Korean Noodle House,"35-04 154th StFlushing, NY 11354",closed now,$$,4.5,Korean,(718) 445-2100,None,No,40.765829,-73.810862
6,Nan Xiang Xiao Long Bao,"38-12 Prince StFlushing, NY 11354",Open now,$,4.0,Shanghainese,(718) 321-3838,nan-xiang.com,No,40.759881,-73.832585
7,Prince Noodle and Cafe 老成都,"4009 Prince StFlushing, NY 11354",Open now,$,4.0,Noodles,(718) 888-9295,None,No,40.758565,-73.831513
8,Lan Zhou Handmade Noodle,"4128 Main StGolden Shopping MallFlushing, NY 1...",closed now,$,4.0,Chinese,(347) 757-8208,None,No,40.757178,-73.829317
9,Flower Brook Mifen House,"44-35 Kissena BlvdFlushing, NY 11355",Open now,$,4.5,Chinese,(516) 345-4145,None,Yes,40.753068,-73.821847


In [ ]:
import folium
m = folium.Map(location=[40.8075355, -73.9625727],zoom_start=14)

In [ ]:
# a marker is created for any restaurant: red for critical situation and orange for not critical
for k in range(len(res)):
    folium.Marker([res['lat'].iloc[k],res['lon'].iloc[k]],popup=res['name'].iloc[k]).add_to(m)
m

In [65]:
#workspace with saved results page
url = 'https://www.yelp.com/search?find_desc=lobster&find_loc=new+york&start=60'
response = requests.get(url)
try:
    results_page = BeautifulSoup(response.content,'lxml')
except:
    print("we didn't get back result page from yelp correctly")
  

[<a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="9DuvvLi4D8pNRx96SH4FDw" href="/adredir?ad_business_id=AMNoZngyjT5wiFxIHuYGiA&amp;campaign_id=m4feyFq9f3E874eCYh3l1A&amp;click_origin=search_results&amp;placement=above_search&amp;redirect_url=https%3A%2F%2Fwww.yelp.com%2Fbiz%2Froclyns-bronx&amp;request_id=48f1ab73aa43f7b3&amp;signature=6532e4143762359db9c627ddbd9cf6e4558c0dec701e889a73bfa7e26417acb3&amp;slot=0"><span>Roclyn’s</span></a>,
 <a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="z0XS1eRHD6uK0PN77FI2kg" href="/biz/bap-new-york?osq=lobster"><span>Bap</span></a>,
 <a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="LEXrAvYPfshHjC0L6UsHpA" href="/biz/raffettos-new-york-2?osq=lobster"><span>Raffetto’s</span></a>,
 <a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="Ze0ZcyeWi7VJm-roBwR1rw" href="/biz/ocean-prime-new-york?osq=lobs

In [68]:
businesses = results_page.find_all('a',{'data-analytics-label':"biz-name"})[1:]
businesses.extend(businesses)
businesses

[<a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="z0XS1eRHD6uK0PN77FI2kg" href="/biz/bap-new-york?osq=lobster"><span>Bap</span></a>,
 <a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="LEXrAvYPfshHjC0L6UsHpA" href="/biz/raffettos-new-york-2?osq=lobster"><span>Raffetto’s</span></a>,
 <a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="Ze0ZcyeWi7VJm-roBwR1rw" href="/biz/ocean-prime-new-york?osq=lobster"><span>Ocean Prime</span></a>,
 <a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="ozrPxTeAtHnhNOG0zY3zXA" href="/biz/the-original-soupman-new-york-7?osq=lobster"><span>The Original Soupman</span></a>,
 <a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="eT6hA-oUP2DjC_1DjBwQcQ" href="/biz/rh-rooftop-restaurant-new-york-3?osq=lobster"><span>RH Rooftop Restaurant</span></a>,
 <a class="biz-name js-an